In [13]:
import tarfile
import json
import bz2

In [247]:
with tarfile.open('data/data.tar', 'r') as tar:
    
    # iterating through all files and directories
    for member in tar.getmembers():
        
        #skipping directories
        if member.isdir():
            continue
        
        #looking 4 target files
        if member.name.endswith('.json.bz2'):
            
            file = tar.extractfile(member)
            noBz2 = bz2.decompress(file.read()).decode('utf-8')
            
            for line in noBz2.splitlines():
                
                tweet = json.loads(line)

                if 'text' not in tweet or 'created_at' not in tweet:
                    continue

                tweetTxt = tweet['text']
                created_at = tweet['created_at']

                # Searching matches
                if "people" or "happy" in tweetTxt:
                    print(tweetTxt)

Bby I can feel your helo..
딩 딩 딩
More ice cream fuck it!! Lls
Why are people giving a fuck bout wha other people think?
"PASARÁS POR MI VIDA SIN SABER QUE PASASTE!" #JoséÁngelBuesa
A bird pooped on @SamSweid :') http://t.co/momNVMZL
Happy bday to these 2 special ladies @LavishLauren @NAEkedTruth
Twitter Bot @MrDisclosure Encourages Investors To Disclose Conflicts In Tweets http://t.co/mnl1WQBm
"@MsHarris4: "@dy_lejoi: "@JazSharee: who tf pawns panties?"  #dead!" Me me me me (jumping wit my hand up)jk"..ewwwwwwwwww lmao
@dujek yeah we'll take it after being down 8 pts with 10 min to go, whew! 3rd in the group would be a big showing for us
しかも外人すらほぼ居ないｗｗｗｗｗｗｗｗｗ　#DraculaHD
帰りゅ 帰りゅ おうちに帰りゅ！
7:05 weg (:
La Diosa Caissa ha sido muy buena conmigo el ultimo mes, debo de corresponderle.
RT @TheLovableWords: Pain makes you stronger, fear makes you braver, heartbreak makes you wiser. So thank the past for a better future. #TLW
@kirakuni2525 絶対ないwそっちのが上よ
@IvanJose_CL dime.
@minkiedoodle hindi. Si 

みんなリハーサルしてるからわたしもリハーサルしようかな！(謎)
Sebel juga ya tnyta punya tmn g konsisten.. *pfiuhhh..
普通の53歳が赤ちゃん言葉使ったらきもいけどウツは可愛いから良い
Segmentación emocional: el caso de las #redessociales http://t.co/BKjFc2TQ
@AL_GOMESXXX ああ、もう週3も5限あるからマヒしてたよw
@ThurayaAlshaqsi HaHaHaaaa!!
@ShinobuYami @_isabellaromero quando eu casar com o jensen você será madrinha USHUHSAUHAUHAHSU e a isete tb que tá na conversa até agora
@mikephillips001 must be serious if the custard creams are out.
@TweetNeishaa stop having sex
RT @chrisbrown: Sometimes it's better to just shut up!
this is my song for real no doubt. said the dj's makin me feel thugged out. As I walk up on the dance flo. we begin to dance slow.
RT @YourAnonNews: RT @EisMC2: @OccupyDenver OCCUPYIN THE SHIT OUTTA IT! http://t.co/SDpWrhj6
Let's keep #BecauseofJoshGroban going strong. #BecauseofJoshGroban we are united :)
まあららぽはスルーだけどねw 今度服でも買いに来るか
ラクーア、ジジババばっかりね。
YESSS got my allowance!!!! xD
【スポーツMSN 【ＭＬＢ】松井「甘い球だから打ちにいった」: 　■アスレチックス・松井「（２安打とも）甘い球だから打ちにいっただけ。ストライクゾー

Nada nessa vida consegue apagar boas lembranças, nem mesmo o tempo.
cheguei em casa tinha uma PIZZA bem pega , derrotei 5 pedaços !
ทองคำลดลง 450 บาท: ทองคำเปิดตลาดเช้านี้ ลดลง450บบาท แท่งขาย24,150รูปพรรณขาย24,550 บาท http://t.co/ClpcWsAZ
http://t.co/pS5j8Rj5
JAJAJA
僕の好きな君、その君が好きな僕。そうやっていつしか僕は僕を大切に思えたよ。
 my head 
lapar gw
RT @swaggovariess: S/o to the NEW Cole fans , it's ok ,you dick riding , but were still fans , right .
Hã??? Ops... Cai no sono aq,mas vot dmorar mais um pouco,enqnto isso estou #SeguindoTodosDeVolta!!!
@fahfon137 คับผม
@manspeaker Good job on the news spot Chad. Knock 'em dead.
@lastika http://t.co/99iPEioy mau ihhhh....
DO U C URSELF IN A CROWDED ROOM DO U THINK SHE'LL SLEEP WITH A VAGIL STITCH Silversun Pickups  http://t.co/C3YGlRqk via @youtube @TWITTER
@rukawa9218 いやー全然うまくいかなかったですね(⌒-⌒; ) でも25分に入れただけよかっです(˘̩̩̩⌣˘̩̩̩)
最近、会社に着いてリポＤを飲むのが日課になっておる…
あんまりよくない傾向だなぁーヽ(￣д￣;)ノ=3=3=3
RT @AlfredoFlores: JAZZY HAS SWAG! POSING LIKE A G!  http://t.co/OPXwtTrt
RT @courTWEET

秋も深まってきた今日この頃ですが、皆様いかがお過ごしでしょうか？
この度番屋赤坂店もfacebookに参入いたしました！かなりコアな情報ものせています！是非ご覧ください！http://t.co/qXmLOB2G
Awkward.
Sepanjang sejarah idup dsni, baru hari ini rambut ga kena potong .. Haha .. Terima kasih mama atas kesalahan potong saat cuti duluu (¬_¬")
#FreeLA : FREE TABLE and/or TV (los feliz): Take one or the other...or take them both. The table has 2 drawers a... http://t.co/1PXz9Pin
@carly44marie ok name some movies funnier...i gotta see them
@TatiieVasquez hahahah la verdad no sé D: y tenés listo tu cerebro!?:D(Y)
@suzune_u @akawa10 くっそ、笑った。可愛いな（
@DiabloNana no lol and im not gonna get sick :)
@souljaLiFEmenT my shit gone be right next time yuh see mhe
RT @XSTROLOGY: #Gemini: Creative, Spontaneous & Optimistic.
RT @WereJustTeenss: Don't cry over the past, it's gone. Don't stress about the future, it hasn't arrived. Live in the present and make i ...
@LoveLikeLiqueur lol well thank you I dont want you too & good cause you are :) ;)
@yukitre 百合もすきですよ！
@Penzilzzz 5555.
RT @PaveloRoc

Retouching Visuals : Image Editing Company India, Photo Retouching Services, Clipping Path, : Outsourcing #ottawa http://t.co/nkPCyzTN
Trimmed and cut . . Yes.
Awww this movie is so cute ; It makes me cry
Wind 3 km/h NNW. Barometer 1010.0 hPa, Rising slowly. Temperature 26.3 °C. Rain today 0.0 mm. Humidity 36%
@EDDOEV le pregunté el sábado
@trackstarsamar bro @KaylaAlves12 is rude!-___-
@KarlaCantu23 Pero lo divertido es que te vas de vacaciones cuando quieras verdaaaa sistaaaa
@yunseak 핵 폭탄이아니라 방사능입니다 멸망보다는 그 고통에 연속으로 인간이 좀 착해지라는 것 그러나 저또한 그 희생자일수 있고 다음에 태어날 신도 그 여파가 크겠죠
RT @a_arminante: Watch Videum's trailer to understand the power of 'Health in Any Language': http://t.co/USwOQLVU
@andylovato @bethzi_reynaa nadamas el viernes.. yo el martes y miercoles haaaa haa #esdeDios ;)
RT @MyspaceTom_: Myspace tom &gt; Mark ZuckerBitch. .... I been had bitches!
@ninnaaax GEFELICITEERD POP! (L)
요호 과학.......ㅠㅠㅠㅠㅠㅠ못쳣다 기가는 원래 그정도밖에안나오니까 상관없지만ㅋㅋㅋ
@yaboigrant not done partying let's get fucked up fu

In [7]:
#!pip install python-dotenv

In [100]:
# from dotenv import dotenv_values
# import requests
# import json

# # loading API keys
# config = dotenv_values(".env")

# bearer_token = config["BEARER_TOKEN"]

# search_url = "https://api.twitter.com/2/tweets/search/recent"

# # Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# # expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
# # query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id', "max_results":"10"}
# query_params = {
#     'query' : '"Higgs Boson" -is:retweet', # -is:retweet -> avoids retweets
#     #"max_results":"100", # number of results we'll get
#     "start_time": "2023-04-24T23:59:00Z",
#     "end_time": "2023-04-28T22:59:00Z"
    
# }

# # Token authentication method
# def bearer_oauth(r):
#     r.headers["Authorization"] = f"Bearer {bearer_token}"
#     r.headers["User-Agent"] = "v2RecentSearchPython"
#     return r

# # Setting the query
# def connect_to_endpoint(url, params):
#     response = requests.get(url, auth=bearer_oauth, params=params) # how we set the request itself with .get
    
#     return response.json()

# # Querying the API
# json_response = connect_to_endpoint(search_url, query_params)

# # Parsing the response
# parsedRes = json.dumps(json_response, indent=4, sort_keys=True, ensure_ascii=True) # indent 4 to look nice indent. ascii to get emoties etc.
# print(parsedRes)
# # Saving the variable holding tweets to a text file
# file = open("tweets.json", "w")
# file.write(parsedRes)
# file.close
# # Appending the other variables to the same text file
# file = open("tweets.json", "a")
# file.write('\n'+parsedRes)
# file.close


{
    "data": [
        {
            "edit_history_tweet_ids": [
                "1652062535862853633"
            ],
            "id": "1652062535862853633",
            "text": "@elonmusk @MuskUniversity The Higgs Boson is the only recorded thing to defy our comprehension of physics."
        },
        {
            "edit_history_tweet_ids": [
                "1652062005136707584"
            ],
            "id": "1652062005136707584",
            "text": "@MuskUniversity How does Higgs Boson theory play into your truth?"
        },
        {
            "edit_history_tweet_ids": [
                "1652058176832040960"
            ],
            "id": "1652058176832040960",
            "text": "@_Higgs_Boson *nods* Don't blame you in the least, and I'm guessing they'll be gone soon...I just really hope the folks that need to see this, DO see it."
        },
        {
            "edit_history_tweet_ids": [
                "1652057651461910528"
            ],
            "id": "1652

<function TextIOWrapper.close()>

In [94]:
#!pip install tweepy
import tweepy

paginator = tweepy.Paginator(
    json_response,                         # The method you want to use    
    exclude=['retweets', 'replies'],       # Some argument for this method
    start_time="1998-12-01T00:00:00Z",     # Some argument for this method
    #max_results=100                        # How many tweets asked per request
)

for tweet in paginator.flatten(limit=250): # Total number of tweets to retrieve
    print(tweet)

with open(r'tweets.txt', 'r') as fp:
    
    # read all lines using readline()
    lines = fp.readlines()
    
    for row in lines:
        # check if string present on a current line
        if row.find('text') != -1:  # find() method returns -1 if the value is not found
            file1 = open("tweetsText.csv", "a")  # append mode
            file1.write(row.lstrip().replace('"text": "', ''))
            file1.close()  

AttributeError: 'dict' object has no attribute '__name__'